In [5]:
import pynvml
import os
import pandas as pd 

def get_gpu_info():
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    print("GPU Count:", device_count)
    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU {i}:")
        print(f"  Total Memory: {info.total/1024/1024} MB")
        print(f"  Free Memory: {info.free/1024/1024} MB")
        print(f"  Used Memory: {info.used/1024/1024} MB")
    pynvml.nvmlShutdown()

get_gpu_info()

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

GPU Count: 4
GPU 0:
  Total Memory: 81920.0 MB
  Free Memory: 39859.5 MB
  Used Memory: 42060.5 MB
GPU 1:
  Total Memory: 81920.0 MB
  Free Memory: 39779.5 MB
  Used Memory: 42140.5 MB
GPU 2:
  Total Memory: 81920.0 MB
  Free Memory: 18115.4375 MB
  Used Memory: 63804.5625 MB
GPU 3:
  Total Memory: 81920.0 MB
  Free Memory: 10811.5 MB
  Used Memory: 71108.5 MB


In [4]:
file_path = '/home/uceerx0/Test_Data.xlsx'

# Load the Excel file uing pandas
test_frame = pd.read_excel(file_path,sheet_name=1, engine='openpyxl')

test_frame

,ID,Prompt,Category,Classification T1,Classification T2,Amount,Expected Response,GPT 4 Chosen Response,GPT 4 Rejected Response,Unnamed: 9
0,1,I used a TFL travel card to travel from home t...,Travel,Policy Violated,Not Reimbursable,0.0,"According to the GNEI Expenses Policy, travel ...","Based on the GNEI Expenses Policy, travel betw...",Based on the GNEI Expenses Policy provided in ...,NaN
1,2,I travelled from London to Amsterdam for a con...,Travel,Policy Not Violated,Fully Reimbursable,95.0,Based on the information provided: \nYou bough...,For the flight ticket from London to Amsterdam...,For your air travel expenses from London to Am...,NaN
2,3,I booked a taxi from the GNEI campus to my hom...,Travel,Policy Not Violated,Fully Reimbursable,30.0,NaN,"Yes, you can expense the taxi charge of £30 fr...","Yes, you can expense the £30 taxi charge for t...",NaN
3,4,I booked a taxi from the GNEI campus to my hom...,Travel,Policy Violated,Further Clarification Required,0.0,NaN,"Based on the GNEI Expenses Policy, taxi fares ...","Based on the GNEI Expenses Policy, taxi fares ...",NaN
4,5,I traveled to the GNEI campus to pick up some ...,Travel,Policy Not Violated,Fully Reimbursable,15.0,NaN,"Yes, you can expense the £15 parking fee incur...","Yes, you can expense the £15 parking fee incur...",NaN
...,...,...,...,...,...,...,...,...,...,...
195,196,I went on a work trip to China and booked an 8...,Accomodation and Sustenance,Policy Not Violated,Fully Reimbursable,215.0,NaN,"For your 8-day work trip to China, where you b...",NaN,NaN
196,197,I went on a work trip to Germany and booked an...,Accomodation and Sustenance,Policy Violated,Further Clarification Required,0.0,NaN,The GNEI Expenses Policy requires electronic i...,NaN,NaN
197,198,I was sent to work at GNEI College in Washingt...,Accomodation and Sustenance,Policy Not Violated,Fully Reimbursable,5500.0,NaN,"For your long-term assignment in Washington, U...",NaN,NaN
198,199,I'm a staff of GNEI Campus in Manchester. I we...,Accomodation and Sustenance,Policy Violated,Fully Reimbursable,210.0,NaN,"For your work trip to London, staying for 6 ni...",NaN,NaN


In [4]:
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import transformers
from langchain import PromptTemplate


client = chromadb.Client()
# if exist,delete
if "document" in [col.name for col in client.list_collections()]:
    client.delete_collection("document")
    print("deleted")

collection = client.create_collection("document")
collection2 = client.create_collection("travel_document")

In [7]:
context_full = """

GNEI Expenses Policy
Summary Guidance
Subject
Page No.
Train Travel
This should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel via GNEI's approved travel provider.


Taxis
Taxis may be used where there is a clear requirement or where they are the most economic practical means of transport.


Business Mileage
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter, Motorcycles - 26p per mile, Bicycles - 22p per mile.


Air Travel
This should be booked using GNEI’s preferred travel provider where possible. For flights of five hours or less, staff should book restricted tickets in economy. For flights lasting five hours or longer staff may travel premium economy. Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class.


Hotel Rates
UK: should not exceed £210 including VAT (room only) in London or £140 including VAT (room only) outside of London. The cost of accommodation at or near the normal place of work wouldn’t qualify as business travel and would therefore be subject to tax.
Overseas: not subject to a fixed rate or limit. Claimants should instead look to apply this policy's general principles (section 3) when assessing reasonableness and use either HMRC's overseas benchmark rates and / or this policy's equivalent UK rates for guidance


Meals whilst away overnight
UK Breakfast: maximum of £12 receipted including VAT and service; unreceipted £5.5
UK Lunch: maximum of £18 receipted including VAT and service; unreceipted £6
UK Dinner: maximum of £36 receipted including VAT and service, unreceipted £17.
UK 24-hour rate: In line with the above, up to £66 receipted (£28 unreceipted).
Overseas: not subject to a fixed rate or limit. Claimants should instead look to apply this policy's general principles (section 3) when assessing reasonableness and use either HMRC's overseas benchmark rates and / or this policy's equivalent UK rates for guidance.


Line Rental
Line rental cannot be claimed; only the cost of business calls on a call-by-call basis can be claimed.


Broadband
Home internet connection cannot be claimed.


Staff and Student Entertaining
This means food or drink for two or more members of staff or registered GNEI students in connection with GNEI business activities. The cost of entertaining should not exceed £22 per head.


Staff Social Functions
Events that are not in connection to GNEI business activities e.g. Christmas lunches/parties, end of term socials, retirement parties etc. The GNEI department contribution should be no more than £22 per head.


Business entertaining
Cost should be appropriate and not exceed £44 per head including alcohol and service unless exceptional circumstances apply.


Professional Subscriptions
GNEI will only reimburse staff, or pay on their behalf, annual subscriptions or memberships to a professional body where either:
Savings to GNEI arising from membership, for example reduced conference attendance fees, exceed the cost of membership, or
Membership is mandatory in order to be able to teach on a professionally accredited course.
In all cases, the professional body must feature in the list of approved organisations published by HMRC.





1. Introduction 
This Expenses Policy (‘the Policy’) provides guidance to all individuals claiming reimbursement of reasonable expenses incurred in connection with GNEI business.
GNEI is a charity and a large recipient of public funds from UKRI and the Office for Students, grants from other public bodies, charities and fees paid by students. All expenditure should be appropriate and modest in scale.
This policy has been prepared in accordance with Income Tax and National Insurance Contribution regulations and Her Majesty’s Revenue and Customs (HMRC) regulations.



2. Scope
This policy applies to all spend on GNEI activities including from research grants and discretionary accounts as well as departmental codes. 
This policy covers the following areas of expenditure:
Travel
Overnight costs and allowances
Telephone and internet costs
Entertainment & Hospitality
Training
Subscriptions
Advances
Other Expenditure
For items that do not fall under any of the above headings, please refer to your local finance team or Accounts Payable at expenses@gnei.ac.uk.
For expense claim procedures, please refer to Annex 1.
Generally, GNEI will not reimburse individuals for:
Broadband/Internet Provision
Equipment
Fines
Gifts
Insurance
Mobile Phone Contracts/Hardware
Personal Expenditure
Professional Subscriptions
Stationery
For further details on the above exclusions, please refer to Annex 2.
Information on VAT aspects which are common to all claims can be found at Annex 3.
This policy also provides information on subsistence payments for visiting Research Fellows, please refer to Annex 4.
Childcare and caring responsibility can be claimed in certain circumstances, please refer to Annex 5.



3. General Principles
GNEI employees and all others engaged in GNEI activity (collectively claimants) will be reimbursed for the actual cost of expenses incurred, wholly, exclusively and necessarily in the performance of GNEI activity as prescribed in this policy.
The expense must be justifiable and reasonable according to the information in this policy, and the claim should always be prepared honestly, legally and responsibly. Any breach of the policy could lead to disciplinary action, up to and including dismissal.
Whilst this policy aims to provide comprehensive guidance on reimbursable out-of- pocket expenditure, it is recommended that staff still seek advance approval from their budget holder or department manager in any situation where interpretation of this policy is in doubt.
Employees should be able to procure the majority of goods and services required for GNEI business using the standard purchase to payment procedures using the GNEIFinance system.
It is the responsibility of authorised signatories to ensure that this expenses policy is upheld; and it should be noted that individual claims falling outside of the policy may be subject to review and/or rejection by Finance.
Electronic images of original receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used should include details of what goods or services have been purchased. Original paper receipts do not need to be stored locally, unless they relate to sponsored research and it is explicit in the terms of the grant that original paper receipts need to be retained.
When making a claim for a group, the most senior person present should pay for the expenditure and make the claim.
Expenses should be submitted as soon as possible after they have been incurred. Claims must be made within three months. Approval should be gained from the relevant School or PS Head of Finance if the claim is over three months old.
It costs a fixed amount to process any claim, so claimants should endeavour not to submit claims for less than £33, unless they represent the total of expenses in a three-month period.
If a claimant inadvertently makes an error with an expense claim and has been reimbursed, GNEI will recover the amount from them.
Expenses must not be used as a way of rewarding people or encouraging them to work in remote locations.
GNEI will not meet the cost of expenses for spouses, partners, other family members or friends of GNEI staff.
Claims are checked by Accounts Payable staff and any deemed to be fraudulent will be investigated and referred to GNEI’s Internal Auditors. If a claimant is found to have submitted a fraudulent claim they will face disciplinary proceedings, leading to penalties up to and including summary dismissal.



4. Travel
This section details what can be claimed as travel expenses and provides information on specific modes of transport and other expenses associated with travel.

General guidance for travelling on GNEI business
The cost of business travel, that is, journeys away from your normal place of work while undertaking GNEI business can be claimed.
In line with the GNEI travel policy, all business trips in the UK or abroad must be authorised in writing in advance by the GNEI budget holder responsible for making the funds available. It is recommended that this is attached to the claim.
The budget holder / manager must agree that the travel is necessary and that alternative methods such as conference call, video conference, phone or email cannot be used or are not appropriate. Consideration should also be given as to how to minimise the environmental impact of travel.
This authorisation ensures that anticipated costs are in line with departmental plans, are eligible under the terms of any associated funding, and that subsequent expense claims can be submitted with confidence, subject to the provisions of this policy.
GNEI has preferred travel suppliers for airline, hotel and car hire bookings details on these suppliers can be found on the Procurement website. These suppliers should be used for travel bookings where possible.

Private travel costs
The following cannot be claimed:
Travel between home and normal place of work.
Business travel broadly similar to the claimant’s normal commute.
Recreational travel and accommodation at or near the business travel destination.
Travel or accommodation for family or friends accompanying the claimant on the business journey.

Rate of exchange
If a credit card was used to pay for overseas expenses, the rate charged by the credit card issuer can be claimed. A copy of the statement should be used to evidence the rate used.
For employees GNEIExpenses will use a default rate for translating foreign currency transactions; this can be overridden if different to the rate at the time of expenditure and fluctuations will be allowed within a set tolerance. If the automatic rate is overridden, a copy of the bank statement should be included with the claim to back up the rate used.
Buying foreign currency is not a genuine business expense and cannot be claimed.

Travel Insurance
Travel Insurance needs to be activated before traveling overseas, travelling in the UK for fieldtrips or travelling on a business trip where there is an overnight stay or air travel involved.
Insurance can be activated by completing a travel insurance request form, which can be found in the Insurance Section on the Finance website.
The Insurance team will issue a cover note and provide details of the policy. There is no cost to departments for travel insurance. For regular travellers the Insurance Section can issue annual cover notes on request.
If a claimant has an existing medical condition and is not travelling against their doctors’ orders, this is covered by the GNEI Travel Insurance Policy at no extra cost.
Personal travel insurance cannot be claimed as GNEI already provides cover for business travel, please refer to Annex 2 for further information on this exclusion.

Public Transport
Journeys made on rail, bus, ferry, river boat, underground, metro or tram services – can be claimed.
The most economical method of travel should be used. For example, if the claimant uses public transport in London regularly, they should use an Oyster card rather than paying for single journeys by cash.

Train journeys
These should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel.
If the claimant does travel first class, an explanation must be included on the claim and if necessary the claim will be passed to the relevant School or PS Head of Finance for approval.
Trains should be booked in advance where possible via GNEI's approved travel provider or similar rail travel provider as this is the most cost effective option. Tickets can then be collected at the relevant station or sent via post.

Tube and bus journeys in London
If a London Transport Oyster or contactless credit/debit card is used for business journeys, the details of each individual journey should be entered on the claim, being specific about locations, date and times and purpose of journey.
Only the relevant journey should be claimed, not the whole top-up cost, unless the whole top-up will be used for GNEI business travel. Receipts are not necessary for individual journeys but are required if claiming for the whole top- up fee.

Receipts
Relevant tickets or documentation should be kept and attached to the expense claim electronically.
Claimants who have a rail or Oyster season ticket enabled to use the underground can only claim for travel if extra expenses are incurred that are not covered by the season ticket and can provide a receipt as proof of the expense.
Ticket machines will provide receipts if the ticket is retained by the automatic barrier at the end of a journey.

Taxis
Taxi fares can only be claimed for journeys where:
It is clearly the most efficient and cost-effective method of transport.
Alternative methods of transport are impractical due to pregnancy, disability, illness or injury, luggage or similar.
A member of staff is working in the office very late, i.e. after 11pm.
Prior approval has been given, where feasible, from the budget holder or departmental administrator/manager.

Business Mileage
If a car, motorcycle or bicycle has been used on a business trip the costs can be claimed using a mileage allowance – that is, a sum of money for every business mile travelled. GNEI does not reimburse claims based on actual petrol receipts.
GNEI has standard rates for these allowances, depending on the circumstances surrounding the trip (see below).
Only additional costs incurred, above what would normally be spent for commuting to work (which are not eligible as a business expense) can be claimed.
A car should only be used if it is cheaper and more efficient than public transport. Cars should be shared with other staff members wherever possible.
The claim should state the start and end points of the journey, along with the number of miles being claimed.
Mileage is paid at the HMRC approved mileage allowance rates:
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter.
Motorcycles - 26p per mile.
Bicycles - 22p per mile.

Car/bicycle/motorcycle insurance
It is important that adequate personal insurance cover for business use is in place, as cars, bicycles or motorcycles belonging to claimants are not covered by GNEI’s insurance. Personal insurance cannot be claimed, see Annex 2 for further information.
If a staff member uses or allows another member of staff to use a vehicle on GNEI’s business that is insured for third party risks only, GNEI will not be held liable for any of the following:
damage or repairs to the vehicle
property lost or damaged
personal injuries sustained or caused during or as a result of such use.

Car sharing
Only the driver can claim mileage for a journey where two or more GNEI staff share a car. The following cannot be claimed:
Insurance cover for private cars.
Accident or breakdown recovery costs, such as AA membership.

Air Travel
Air travel should be booked using GNEI’s preferred travel provider where possible.
GNEI prefers lower fares, based on flight arrival and departure times which cannot be changed. Where there is a need for flexibility in departure or arrival times, a different ticket option will have to be requested. Flexible tickets for flights under three hours require approval by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.
Membership of a frequent traveller scheme and potential air miles benefits must not deflect from using the most cost-effective airline option for travel.
GNEI does not meet the travelling costs of:
Employees of contractors
Spouses, partners, other family members or friends.
Classes of travel:
For flights of five hours or less, staff should book restricted tickets in economy.
For flights lasting five hours or longer staff may travel premium economy.
Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class. This must be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Toll charges
Toll charges paid for in the course of a business journey can be claimed. Toll bridges or roads rarely give receipts; therefore the name of the toll bridge or road should be included on the expense claim.

Congestion charges
GNEI will not reimburse the cost of any congestion charge for staff or visitors except in exceptional circumstances, e.g. where a member of staff has to collect/deliver equipment to a location and the journey could not be completed on public transport. This must be approved in advance by the relevant School/PS Head of Finance.

Car Parking
Reasonable parking costs on business visits and journeys away from the office can be claimed.
Hotel parking costs can be claimed if they are charged separately on the bill.
If a claimant has to drive to their place of work as part of a business journey, e.g. to collect equipment, parking costs can be claimed. Details of the reason should be included with the claim.
The cost of parking at an entertainment venue cannot be claimed.
VAT can be reclaimed (where it's charged) on parking fees which cost less than £28 even if the claimant is not able to get a VAT receipt.

Car Hire
Approval must be obtained from the budget holder or departmental administrator/manager before a car is hired.
If hiring a car is the cheapest mode of travel, it should be booked through GNEI's approved car hire provider (for UK or overseas).
The car should only be used for business purposes.
Rates are inclusive of CDW (Collision Damage Waiver), TPI (Third Party Insurance) and PAI (Personal Accident Insurance).
Any petrol bought solely for business purposes can be claimed. A VAT receipt should be obtained for purchases, a credit card receipt is not sufficient.

Group Travel
This is in relation to a staff conference or student field trip. Prior approval from the budget holder or departmental administrator/manager should be obtained before booking a group trip.
The same rules apply for each category of travel and overnight costs as for travelling as an individual.
All travel (including coach hire) and overnight costs should be booked in advance.
GNEI's approved travel provider provides a group travel & conference booking facility, where all elements of the trip can be arranged. This can be used for arranging a business conference, as well as student field trips.
Please ensure adequate insurance is in place for all staff and students on the trip. Travel insurance needs to be activated prior to the trip by using the Insurance section of the Finance website.

Travel Incidentals
Claimants can claim the following goods or services if they are related to the business purpose of the trip:
The cost of obtaining a visa for working abroad on GNEI business. These can be obtained from GNEI's approved travel provider.
The cost of any vaccinations needed for working abroad.
There may be occasions when a claimant needs a second passport, for example when travel to one country could lead to immigration difficulties in travelling on business to some other countries. GNEI will pay this expense.
Approval must be obtained in advance from the budget holder or departmental administrator/manager.



5. Overnight costs and allowances
Such costs are subject to the requirement for pre-approval in section 4.3 above.

Hotels
In line with the GNEI travel policy, Hotels must be booked via GNEI's approved travel provider for both overseas and UK travel, with recovery through expenses only permitted in those cases set out in Section 3.1 of that policy and subject to an appropriate level of advance approval as set out in section 3.2. The approval obtained must be attached to submitted expense claims alongside receipts for the room cost, breakfast, dinner and other eligible incidentals.
Hotel city tax: from time to time, additional local city tax charges may apply for accommodation. These can be reclaimed via expenses.

UK Hotels
As stipulated by HMRC, the cost of accommodation at or near the normal place of work does not qualify as business travel; any claim would be subject to tax on the individual as a reimbursement of personal expenditure. Such costs may not be claimed, and staff are asked to bear this in mind when making their travel arrangements.
The cost of a hotel in the UK should not exceed £210 including VAT (room only) in London or £140 including VAT (room only) outside of London. If a hotel cost is higher than these limits, this should be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Overseas hotels
The cost of overseas accommodation may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this in mind, the University does not look to prescribe fixed limits for overseas accommodation but instead provide guidance on what a reasonable claim would be so that claimants and approvers are able to make a fully informed assessment.
In the first instance, employees should expect to be reimbursed for hotels that are equivalent to the most appropriate GBP rate for UK travel detailed in section 5.5. For overseas stays the expectation of the University is that an employee should book accommodation that is equivalent to a UK 3-star level. For areas where there is deprivation or a high security risk then the rating may be increased to 4-star.
Employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit nor an automatic entitlement but should be considered a point of reference only.
Employees should be aware however that claims materially in excess of the HMRC rates are likely to be subject to challenge either during the approval process or before settlement by the central processing teams. Where this is expected to be the case, claimants should ensure that clear justification and confirmation of pre-approval from the relevant budget holder is attached to the claim as per Section 4.3.
Where attendance at an international conference requires specific accommodation (for example by virtue of a fixed attendance package or as a necessary means of realising the benefits of the conference), this factor should also be documented with pre-approval attached to the claim.
Incidentals
All necessary incidentals (including brief personal calls home) should be receipted and claimed:
Newspapers, bar drinks, mini bar, hotel video and health and fitness facilities cannot be claimed.
Laundry costs can only be claimed if absolutely necessary, the cost is reasonable and appropriate and the stay away is for at least 5 consecutive nights.

Telephone, Fax and Internet Charges
The cost of business calls, fax and internet access charges for business use, as long as they are included on an itemised bill, can be claimed.
When working overseas a telephone card can be purchased as this reduces the cost of local calls, the cost of this instead of calls from hotels can be claimed.

Meals
When travelling on GNEI business, employees are entitled to recover the cost of reasonable subsistence, namely breakfast, lunch, and dinner. Meals cannot be claimed if already included in hotel accommodation rates being claimed for.
If several members of staff have a meal together, the most senior person should pay and submit the expense claim. In such a case no-one other than the most senior person should claim an allowance for the meals. Please ensure that it is made clear in the justification field of GNEIExpenses that the receipt is covering more than one employee’s meal, and each employee name (covered by the receipt) should be listed.
The following rates include the cost of alcoholic beverages.

UK Meals
Breakfast. If breakfast is not included in the hotel room rate, the cost can be claimed, which should be receipted and claimed as part of the overnight stay to a maximum of £12 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for breakfast.
Lunch. If the claimant is working away from their normal place of work, lunch can be claimed to a maximum of £18 including VAT and service. If it is not possible to obtain a receipt, then £5.5 may be claimed for lunch.
Evening meal. The cost of an evening meal can be claimed which should be receipted and claimed as part of the overnight stay to a maximum of £36 including VAT and service. If it is not possible to obtain a receipt, then £17 may be claimed for dinner.
24-hour rate: Where UK travel includes an overnight stay and incorporates breakfast, lunch and dinner a single claim of up to £66 receipted (£28 un-receipted) may be made. In these cases, individual meal rates need not be applied, and the claim will be assessed on an aggregate basis in order to allow for greater flexibility. This 24-hour rate will not apply where any one of the meals is included within a room rate, and claimants must still provide receipts for each separate meal. Only actual spend will be reimbursed and all claims remain subject to the overriding principles of reasonableness and value for money.

Overseas Meals
The cost of meals may vary significantly from country to country. Both claimants and approvers should look to the general principles of this policy therefore when incurring such costs and when determining an appropriate level of claim: in all respects expenses must be justifiable, reasonable and represent value for money.
With this mind, the University does not look to prescribe fixed limits for overseas meals but instead provide guidance on what a reasonable claim would be.
In the first instance, employees should expect to be reimbursed for meals that are equivalent to the standard of meal that can be purchased in the UK within the set GBP rates for UK travel detailed in sections 5.18 to 21.
As an additional location-specific guide, employees may also consult HMRC's scale rates for overseas travel in order to judge what is reasonable when quoted in local currency. These rates do not represent an upper limit for claims (noting that they’re updated infrequently) but nor do they represent an automatic entitlement. The rates should be considered a point of reference only and all claims must uphold the general principles set out above.
In line with section 3 above, employees will only be reimbursed for actual costs incurred as evidenced by original receipts. Where receipts for overseas subsistence cannot be obtained, employees may claim at the unreceipted GBP rates set out in sections 5.18 to 5.21 up to a maximum of £28 per day for three meals.

Rented Accommodation (This section applies to GNEI staff and students only, the policy for visiting Research Fellows is detailed in Annex 4).
Long-term assignments
If a member of staff is working away from home on a longer-term assignment – usually for six months or more – the budget holder may require them to rent a property rather than stay in a hotel.
The lease should be taken out in the employee’s name as they will be the sole occupant of the property; prior approval from budget holder or departmental administrator/manager must be obtained before the lease is agreed.
The monthly rent, council tax, service charges, TV licence and utility bills (including telephone line rental) including VAT can be claimed. Alternatively, GNEI can pay the rent directly on invoice via Accounts Payable.
If several colleagues are working away from home working on the same project and sharing a property, only one of the staff should claim expenses for the cost of the accommodation and associated bills.

Relocation Expenses
Please refer to the Relocation Expenses Guide for further details.



6. Telephone and Internet costs
This category details what can be claimed when using a telephone for business purposes.
Always use the cheaper of either mobile or land line.

GNEI mobile phone users
As calls and line rental are paid directly by GNEI, no claims for business calls, from either a personal mobile or a home landline should be made by staff in receipt of a GNEI mobile phone.
If it is decided that an employee requires a mobile phone for work purposes, it should be obtained via ISD’s Managed Mobile service.

Own mobile phone users
Line rental cannot be claimed. Only the cost of business calls on a call-by-call basis can be claimed, and an itemised VAT bill with the relevant calls highlighted must be provided as a receipt and attached to the claim.

Home personal land line
Only the cost of business calls on a call-by-call basis can be claimed, and an itemised VAT bill with the relevant calls highlighted must be provided as a receipt and attached to the claim.
Any telephone line rental or standing charges cannot be claimed.

Wi-Fi
Short Wi-Fi internet session fees e.g. at airports and hotels while on GNEI business can be claimed, a receipt must be provided.

Broadband/Internet Provision
Home internet connection cannot be claimed for in any circumstances.



7. Entertainment and Hospitality
This section details what can be claimed when entertaining staff, students or external visitors – that is, providing food, drink or other hospitality.
The nature of the event should be established:
Staff and student entertaining: the event is to primarily entertain GNEI staff and students with some (or no) visitors.
Business entertaining: the event is to primarily entertain visitors with some GNEI staff and students attending.
A record of all staff and student attendees and their GNEI department names must be included with any claim – this is an HMRC requirement.
All expenditure should be reasonable and appropriate.
The cost of entertainment and / or hospitality should not be paid for using departmental purchasing cards under any circumstances.

Hospitality for meetings
Food can only be provided when it is necessary to hold a meeting during a normal mealtime (breakfast - before 9.00am, lunchtime - between 12.00 and 2.00pm, or evening - after 6.00pm), and it is not practical to expect staff to bring their own food. No alcohol can be provided.
Where possible the meeting should be held on GNEI premises and food provided by GNEI's catering providers.

Staff and student entertaining/hospitality
This means food or drink for two or more members of staff or registered GNEI students in connection to GNEI business activities. It includes working lunches/dinners and team- building events. Approval from the budget holder or departmental administrator/manager must be sought before organising or paying for any staff or student entertaining.
The cost of entertaining should not exceed £22 per head.

Staff social functions
These are events that not in connection to GNEI business activities e.g. Christmas lunches/parties, end of term socials, retirement parties etc.
In general, staff should be expected to contribute to the cost of these events. The GNEI department contribution should be no more than £22 per head.
Any expenditure on social functions should be approved in advance with the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance.
Costs for major staff entertaining functions (for example annual social) should be invoiced directly to GNEI, not entered on individuals' expense claims.

Business entertaining
The most senior person present should pay the bill for any business entertaining and submit the claim.
Hospitality, where feasible, should be pre-approved by the budget holder or departmental administrator/manager.
The cost should be appropriate and not exceed £44 per head including alcohol and service.
If for an exceptional reason the costs exceeds £44 per head, including alcohol and service, this should be approved by the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance.
A record of all attendees and their organisation names must be kept and included with the claim; this includes any GNEI staff or students attending.
GNEI staff or students attending a business entertaining event in a support role, e.g. greeting guests, without participating in the hospitality should not be included in the numbers attending. Treat any costs arising from their attendance as part of the cost of the function and claim them as such. 



8. Training
Training and study costs can only be claimed if the training is for a business purpose.

External training courses
External training courses should be booked using a GNEI Purchase Order and paid directly by GNEI on invoice via Accounts Payable.
Prior approval for the expense from the budget holder or departmental administrator/manager must be obtained before booking the course.

Study Assistance
The GNEI Study Assistance Scheme (SAS) provides a range of support for GNEI staff seeking to gain qualifications to support their work and careers.



9. Other Expenditure
Professional body subscriptions
GNEI will only reimburse staff, or pay on their behalf, annual subscriptions or memberships to a professional body where either there are:
Savings to GNEI arising from membership, for example reduced conference attendance fees, exceed the cost of membership, or
Membership is mandatory in order to be able to teach on a professionally accredited course.
In all cases, the professional body must feature in the list of approved organisations published by HMRC.

Books, journals and magazines/periodicals
Most books and journals should be purchased via a GNEI PO and paid directly by GNEI on invoice via Accounts Payable.
Books should only be purchased directly by a member of staff when this is not possible, e.g. Amazon does not accept purchase orders.

UK Work Permit (including renewal)
In exceptional circumstances, and with prior approval from the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance, the cost of obtaining or renewing a visa for working in the UK may be claimed through expenses.

Specialist clothing
If a job requires specialist clothing, this should usually be purchased via a GNEI PO and paid directly by GNEI on an invoice via Accounts Payable.

Subject Fees
Where individuals are paid cash to take part in a research activity/survey, the individual is required to sign a form stating that they have received the cash from the claimant. This form should be attached to the expenses claim form as a receipt.



10. Advances
Foreign Advances
These are available for overseas travel on behalf of GNEI where other means of paying for the trip are not available.
Staff may apply for foreign currency, traveller’s cheques or for the sterling equivalent to be paid directly into their bank account. Please go to the Foreign Advances Policy for further information.

Research Advances for Subject Fees
Staff may apply through their departments for a Research Advance for the purpose of remunerating Subjects for taking part in experiments, to a maximum of £55 cash per subject for the duration of the experiment.
Subjects may claim in addition to this their travel (where allowed by the grant) from the advance on the production of the original receipt.
If the Subject is to be paid more than £55 (not including travel) or they will be taking part in multiple experiments within the same research where the total is more than £55, this should be paid to the Subject directly by GNEI using a general expense claim form and not through the advance (due to HMRC tax regulations). Please go to the Research Advances Policy for further information.




Annex 1 – Expense claim procedures
How to make a claim
All GNEI employees and students with a UK bank account must claim expenses online via the GNEIExpenses module on GNEIFinance. Anyone else should use the external visitor system.

Receipts
For GNEI employees, electronic images of original VAT receipts must be attached to all claims. Once submitted each claim will be routed automatically to the appropriate approver(s) and Accounts Payable.

For students or external individuals, a claim will be processed once the approved claim form and all supporting paper receipts have reached Accounts Payable.

Please see Annex 3 for further information on VAT receipts. Credit card slips or credit card/bank statements will not be accepted as evidence of business expenditure. All receipts used must include details of the goods or services purchased. 

If the necessary receipts or documentation is not available claimants should obtain approval from the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance prior to submitting the claim, and provide evidence of this approval attached to the claim. Use the Justification field in GNEIExpenses to notify Accounts Payable that the approval is attached.

Approval
GNEI operates a system of delegated authorisation to staff and approval limits are set out in the GNEI Financial Regulations. Expense approvers should be aware of the funds available in the project to which the claim is being charged to.

All expense claims require authorisation prior to payment. This authorisation is in line with GNEI’s Financial Authorised Signatory hierarchy. It is the responsibility of the claimant to ensure that the approver will confirm authorisation for each item in the claim. Claimants may be personally liable for expenses not subsequently approved.

Approver(s) must personally review and then approve the claim. Approvers are expected to review claims promptly and consider the following before approving:
The appropriateness and quantum of the expenditure
The expenses category chosen for each item
The project code(s) charged
Any policy violations
That receipts or receipt images correspond to the claim and constitute itemised receipts, not credit card slips/bank statements.
That items within the claim meet the terms of the research funder’s terms and
conditions (if related to sponsored research).

If there are queries on any of the above, claims should be challenged by the Approver and
rejected if incorrect.

Expense claims not appropriately authorised will be returned.




Annex 2 – Exclusions
In exceptional circumstances claims for some of these items may be allowed, prior approval should be obtained from the relevant PS Director, Faculty Director of Operations or School/PS Head of Finance. If prior approval is not obtained there is no guarantee that expenditure will be reimbursed.

Broadband/Internet Provision
Home internet connection cannot be claimed for in any circumstances.

Entertainment and Hospitality
GNEI will not pay for:
Clothing for entertaining e.g. dress or suit hire
Overnight accommodation

Equipment
All equipment should be purchased directly by GNEI via a Purchase Order (PO) and paid directly on invoice via Accounts Payable. Categories of equipment include, although not exclusively, IT hardware & software, laboratory equipment & consumables. GNEI have negotiated agreements with Contracted Suppliers for equipment.

Fines
GNEI does not pay:
Fines or fixed penalties
Administration fees charged by third parties for recovering fines
Fines for motoring offences

Gifts
GNEI will not pay for gifts in any circumstances. Gifts for staff should be purchased using personal funds or via a staff collection. Gifts should not be claimed via expenses, nor purchased via iProcurement or a GNEI Purchasing Card.

Insurance
Travel insurance should be obtained from the Insurance section of the GNEI Finance website. Personal travel insurance policies will not cover claimants whilst abroad on GNEI’s behalf, and cannot be claimed as GNEI already provides cover for business travel. If a claim is received for travel insurance it will not be paid. Personal car/bicycle/motorcycle insurance cannot be claimed.

Mobile Phone Contracts/Hardware
Monthly contract charges and hardware cannot be claimed for. If a mobile phone is required for work purposes it should be obtained from ISD Telecoms. 
GNEI will pay for the cost of business calls made from a personal mobile phone, please see the ‘Telephone costs’ section of the GNEI Expenses Policy.

Personal Expenditure
Personal incidental costs while away on GNEI activity cannot be claimed. These include, although not exclusively, newspapers, bar drinks, mini bar, hotel video and health and fitness facilities.

Stationery
All stationery must be purchased via GNEI's approved supplier, via a PO and paid directly on invoice via Accounts Payable.




Annex 3 – VAT
VAT
GNEI can reclaim the VAT charged on some business expenses. In order to make a proper claim GNEI needs valid receipts to provide appropriate evidence. It is the claimant’s responsibility to obtain a receipt showing VAT where it applies.

As defined by HMRC, original VAT receipts must accompany all claims. Credit card slips or credit card/bank statements will not be accepted as evidence of expenditure. All receipts should show the following information:
Name, address and VAT number of the supplier
Date of supply and date of issue if different
Description of the goods or services
For each VAT rate the total amount payable including VAT and the VAT rate charged
Name and address of person to whom the services are supplied (if appropriate)
Unit price (if the supply cannot be broken down into countable elements then the total tax exclusive price will be the unit price)
The total amount of VAT charged



Annex 4 - Visiting Research Fellows Subsistence Payments
Subsistence payments should be claimed via a Visiting Research Fellow form, which will be processed as a manual  expense form if the following conditions are met:
The general rule is to determine if an overseas visitor is an employee or not. If the overseas visitor does not undertake any duties (other than research) normally performed by GNEI staff and they are not subject to the usual terms and conditions of employment, then payments may be made to them without deduction of PAYE and NI.
Usual terms and conditions of employment would include such matters as the ability of GNEI to determine hours worked, location of work, to set a probationary period and entitlement of the payee to holiday pay, sick pay and to join a pension scheme.

The individual must be visiting for less than two years. 

Subsistence payments may be made gross on the Visiting Research Fellow form if the following conditions are met:
There is no contract of employment.
There is no contract for services, whether written or verbal.
The visiting Research Fellow does not undertake any duties (apart from research) normally performed by staff of GNEI.
The grant is paid to cover living and travel expenses.
The visiting Research Fellow is here for a maximum of two years.
The visiting Research Fellow form must clearly state that the payment is for travel and subsistence of an overseas visiting researcher.
The individual’s home country and institution address is included on the form.
The source of funding is stated on the form.
If the visiting Research Fellow does undertake any teaching duties or hold seminars, then a proportion of the payments made to them must be attributed to these duties. Payments may still be made gross but they must be reported to the HMRC. Details of all such payments in any tax year (6 April to 5 April) must be returned to the Taxation and Commercial Accountant in Finance by 30 April each year.


Annex 5 - Childcare and Caring Responsibility costs
Various funders now allow childcare or caring responsibility costs to be claimed as allowable expenses if the costs are incurred for working outside normal working patterns (e.g. childcare or eldercare costs while the primary carer attends a conference outside usual contracted hours).

Please refer to your funder’s terms and conditions or obtain permission from them prior to incurring the expenditure. If you require further guidance please contact Award Services, Research and Innovation Services award-services@gnei.ac.uk

Criteria
The following criteria will need to be met for claims to be approved through GNEIExpenses:
Staff must be on a grant under the terms of which such a claim is expressly allowable and eligible for reimbursement.
There must be an auditable record such as conference attendance confirmation and receipts for the cost of the childcare / caring responsibility.




"""

print(len(context_full))

travel = """

4. Travel
This section details what can be claimed as travel expenses and provides information on specific modes of transport and other expenses associated with travel.

General guidance for travelling on GNEI business
The cost of business travel, that is, journeys away from your normal place of work while undertaking GNEI business can be claimed.
In line with the GNEI travel policy, all business trips in the UK or abroad must be authorised in writing in advance by the GNEI budget holder responsible for making the funds available. It is recommended that this is attached to the claim.
The budget holder / manager must agree that the travel is necessary and that alternative methods such as conference call, video conference, phone or email cannot be used or are not appropriate. Consideration should also be given as to how to minimise the environmental impact of travel.
This authorisation ensures that anticipated costs are in line with departmental plans, are eligible under the terms of any associated funding, and that subsequent expense claims can be submitted with confidence, subject to the provisions of this policy.
GNEI has preferred travel suppliers for airline, hotel and car hire bookings details on these suppliers can be found on the Procurement website. These suppliers should be used for travel bookings where possible.

Private travel costs
The following cannot be claimed:
Travel between home and normal place of work.
Business travel broadly similar to the claimant’s normal commute.
Recreational travel and accommodation at or near the business travel destination.
Travel or accommodation for family or friends accompanying the claimant on the business journey.

Rate of exchange
If a credit card was used to pay for overseas expenses, the rate charged by the credit card issuer can be claimed. A copy of the statement should be used to evidence the rate used.
For employees GNEIExpenses will use a default rate for translating foreign currency transactions; this can be overridden if different to the rate at the time of expenditure and fluctuations will be allowed within a set tolerance. If the automatic rate is overridden, a copy of the bank statement should be included with the claim to back up the rate used.
Buying foreign currency is not a genuine business expense and cannot be claimed.

Travel Insurance
Travel Insurance needs to be activated before traveling overseas, travelling in the UK for fieldtrips or travelling on a business trip where there is an overnight stay or air travel involved.
Insurance can be activated by completing a travel insurance request form, which can be found in the Insurance Section on the Finance website.
The Insurance team will issue a cover note and provide details of the policy. There is no cost to departments for travel insurance. For regular travellers the Insurance Section can issue annual cover notes on request.
If a claimant has an existing medical condition and is not travelling against their doctors’ orders, this is covered by the GNEI Travel Insurance Policy at no extra cost.
Personal travel insurance cannot be claimed as GNEI already provides cover for business travel, please refer to Annex 2 for further information on this exclusion.

Public Transport
Journeys made on rail, bus, ferry, river boat, underground, metro or tram services – can be claimed.
The most economical method of travel should be used. For example, if the claimant uses public transport in London regularly, they should use an Oyster card rather than paying for single journeys by cash.

Train journeys
These should be standard class unless a heavily discounted first-class ticket is booked significantly in advance of the date of travel.
If the claimant does travel first class, an explanation must be included on the claim and if necessary the claim will be passed to the relevant School or PS Head of Finance for approval.
Trains should be booked in advance where possible via GNEI's approved travel provider or similar rail travel provider as this is the most cost effective option. Tickets can then be collected at the relevant station or sent via post.

Tube and bus journeys in London
If a London Transport Oyster or contactless credit/debit card is used for business journeys, the details of each individual journey should be entered on the claim, being specific about locations, date and times and purpose of journey.
Only the relevant journey should be claimed, not the whole top-up cost, unless the whole top-up will be used for GNEI business travel. Receipts are not necessary for individual journeys but are required if claiming for the whole top- up fee.

Receipts
Relevant tickets or documentation should be kept and attached to the expense claim electronically.
Claimants who have a rail or Oyster season ticket enabled to use the underground can only claim for travel if extra expenses are incurred that are not covered by the season ticket and can provide a receipt as proof of the expense.
Ticket machines will provide receipts if the ticket is retained by the automatic barrier at the end of a journey.

Taxis
Taxi fares can only be claimed for journeys where:
It is clearly the most efficient and cost-effective method of transport.
Alternative methods of transport are impractical due to pregnancy, disability, illness or injury, luggage or similar.
A member of staff is working in the office very late, i.e. after 11pm.
Prior approval has been given, where feasible, from the budget holder or departmental administrator/manager.

Business Mileage
If a car, motorcycle or bicycle has been used on a business trip the costs can be claimed using a mileage allowance – that is, a sum of money for every business mile travelled. GNEI does not reimburse claims based on actual petrol receipts.
GNEI has standard rates for these allowances, depending on the circumstances surrounding the trip (see below).
Only additional costs incurred, above what would normally be spent for commuting to work (which are not eligible as a business expense) can be claimed.
A car should only be used if it is cheaper and more efficient than public transport. Cars should be shared with other staff members wherever possible.
The claim should state the start and end points of the journey, along with the number of miles being claimed.
Mileage is paid at the HMRC approved mileage allowance rates:
Cars - 50p per mile for the first 11,000 miles in a tax year and 28p per mile thereafter.
Motorcycles - 26p per mile.
Bicycles - 22p per mile.

Car/bicycle/motorcycle insurance
It is important that adequate personal insurance cover for business use is in place, as cars, bicycles or motorcycles belonging to claimants are not covered by GNEI’s insurance. Personal insurance cannot be claimed, see Annex 2 for further information.
If a staff member uses or allows another member of staff to use a vehicle on GNEI’s business that is insured for third party risks only, GNEI will not be held liable for any of the following:
damage or repairs to the vehicle
property lost or damaged
personal injuries sustained or caused during or as a result of such use.

Car sharing
Only the driver can claim mileage for a journey where two or more GNEI staff share a car. The following cannot be claimed:
Insurance cover for private cars.
Accident or breakdown recovery costs, such as AA membership.

Air Travel
Air travel should be booked using GNEI’s preferred travel provider where possible.
GNEI prefers lower fares, based on flight arrival and departure times which cannot be changed. Where there is a need for flexibility in departure or arrival times, a different ticket option will have to be requested. Flexible tickets for flights under three hours require approval by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.
Membership of a frequent traveller scheme and potential air miles benefits must not deflect from using the most cost-effective airline option for travel.
GNEI does not meet the travelling costs of:
Employees of contractors
Spouses, partners, other family members or friends.
Classes of travel:
For flights of five hours or less, staff should book restricted tickets in economy.
For flights lasting five hours or longer staff may travel premium economy.
Where a long international flight is immediately followed by a presentation or meeting, staff may travel by business class. This must be pre-approved by the relevant Dean / PS Director, Faculty Director of Operations or School/PS Head of Finance.

Toll charges
Toll charges paid for in the course of a business journey can be claimed. Toll bridges or roads rarely give receipts; therefore the name of the toll bridge or road should be included on the expense claim.

Congestion charges
GNEI will not reimburse the cost of any congestion charge for staff or visitors except in exceptional circumstances, e.g. where a member of staff has to collect/deliver equipment to a location and the journey could not be completed on public transport. This must be approved in advance by the relevant School/PS Head of Finance.

Car Parking
Reasonable parking costs on business visits and journeys away from the office can be claimed.
Hotel parking costs can be claimed if they are charged separately on the bill.
If a claimant has to drive to their place of work as part of a business journey, e.g. to collect equipment, parking costs can be claimed. Details of the reason should be included with the claim.
The cost of parking at an entertainment venue cannot be claimed.
VAT can be reclaimed (where it's charged) on parking fees which cost less than £28 even if the claimant is not able to get a VAT receipt.

Car Hire
Approval must be obtained from the budget holder or departmental administrator/manager before a car is hired.
If hiring a car is the cheapest mode of travel, it should be booked through GNEI's approved car hire provider (for UK or overseas).
The car should only be used for business purposes.
Rates are inclusive of CDW (Collision Damage Waiver), TPI (Third Party Insurance) and PAI (Personal Accident Insurance).
Any petrol bought solely for business purposes can be claimed. A VAT receipt should be obtained for purchases, a credit card receipt is not sufficient.

Group Travel
This is in relation to a staff conference or student field trip. Prior approval from the budget holder or departmental administrator/manager should be obtained before booking a group trip.
The same rules apply for each category of travel and overnight costs as for travelling as an individual.
All travel (including coach hire) and overnight costs should be booked in advance.
GNEI's approved travel provider provides a group travel & conference booking facility, where all elements of the trip can be arranged. This can be used for arranging a business conference, as well as student field trips.
Please ensure adequate insurance is in place for all staff and students on the trip. Travel insurance needs to be activated prior to the trip by using the Insurance section of the Finance website.

Travel Incidentals
Claimants can claim the following goods or services if they are related to the business purpose of the trip:
The cost of obtaining a visa for working abroad on GNEI business. These can be obtained from GNEI's approved travel provider.
The cost of any vaccinations needed for working abroad.
There may be occasions when a claimant needs a second passport, for example when travel to one country could lead to immigration difficulties in travelling on business to some other countries. GNEI will pay this expense.
Approval must be obtained in advance from the budget holder or departmental administrator/manager.

"""

print(len(travel))

44769
11691


In [6]:
import re
segments = re.split(r'\n\s*\n', context_full.strip())

# # print segment
# for segment in segments:
#     print(segment)
#     print('-' * 80) 
    
travel_segments = re.split(r'\n\s*\n', travel.strip())

for segment in travel_segments:
    print(segment)
    print('-' * 80)

4. Travel
This section details what can be claimed as travel expenses and provides information on specific modes of transport and other expenses associated with travel.
--------------------------------------------------------------------------------
General guidance for travelling on GNEI business
The cost of business travel, that is, journeys away from your normal place of work while undertaking GNEI business can be claimed.
In line with the GNEI travel policy, all business trips in the UK or abroad must be authorised in writing in advance by the GNEI budget holder responsible for making the funds available. It is recommended that this is attached to the claim.
The budget holder / manager must agree that the travel is necessary and that alternative methods such as conference call, video conference, phone or email cannot be used or are not appropriate. Consideration should also be given as to how to minimise the environmental impact of travel.
This authorisation ensures that anticipate

In [7]:
documents = segments

# select embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)


# embed document into vector
document_embeddings = embedding_model.encode(documents)

# insert vector into ChromaDB
for i, embedding in enumerate(document_embeddings):
    collection.add(ids=[str(i)], documents=[documents[i]], embeddings=[embedding.tolist()])

# initialize HuggingFace Embeddings
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)
# create Chroma vector store
vector_store = Chroma(
    collection_name="document",
    embedding_function=hf_embeddings,
    client=client 
)



########################
# travel part


travel_documents = travel_segments

# embed document into vector
travel_document_embeddings = embedding_model.encode(travel_documents)

# insert vector into ChromaDB
for i, embedding in enumerate(travel_document_embeddings):
    collection2.add(ids=[str(i)], documents=[travel_documents[i]], embeddings=[embedding.tolist()])

# initialize HuggingFace Embeddings
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)
# create Chroma vector store
travel_vector_store = Chroma(
    collection_name="travel_document",
    embedding_function=hf_embeddings,
    client=client 
)

/home/uceerx0/p3-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/uceerx0/p3-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
retriever = vector_store.as_retriever(search_kwargs={"k": 15})

travel_retriever = travel_vector_store.as_retriever(search_kwargs={"k": 10}) 

scenario = test_frame['Prompt'][1]

print(scenario)

results = travel_retriever.get_relevant_documents(scenario)



relevant_texts = [result.page_content for result in results]
length = 0
for text in relevant_texts:
    length = length + len(text)
    print(text)
    print('-' * 80)

print("Length of related document: ", length)

# Create Prompt
template = """The following pieces of context is the expense policy of GNEI related to the scenario at the end. 
If you don't know the answer, just say that you don't know.
Don't try to make up an answer.
<<<<<Policy Start: \n 
{context} 
\n\n\nPolicy End>>>>>\n

Question: 
The related <<<POLICY>>> is provided at the beginning, and a <<<SCENARIO>>> will be provided. you are GREAT at analysing the <<<SCENARIO>>> according to the <<<POLICY>>>. Your answer MUST and and CAN ONLY be a valid json format, having 4 text fields: Classsifcation T1, Reimbursement Amount, Classsifcation T2, and Reasons. Your answer should follow these structures: \n 1) "Classsifcation T1" :you MUST choose the activity in the <<<scenario>>> is 'Policy Violated' or 'Policy not Violated'\n \n 2) "Reimbursement Amount" : Answer according to the Policy the amount of money can be reimbursed totally \n 3) "Classsifcation T2" : Compare to the requested reimbursement amount in the scenario, and answer if the claim is 'Fully Reimbursable'/'Partially Reimbursable'/'Not Reimbursable'/'Further Clarification Required'. \n 4) give reasons for your choices according to the <<<POLICY>>>.\n\n

        <<<<<start examples:

        <<<<<example one:
        scenario: I used a TFL travel card to travel from home to the GNEI work station every day for a week and it cost me £50. How much can I reimburse?

        Answer:{{"Classifcation T1" : "Policy Violated",
                "Reimbursement Amount" : 0£,
                "Classsifcation T2" : "Not Reimbursable",
                "Reasons":"According to the GNEI Expenses Policy, travel between home and normal place of work (i.e., the GNEI campus) cannot be claimed. Thus, this expense cannot be reimbursed."}};

        <<<<<example two:
        scenario: I booked a taxi from the GNEI work station to my home yesterday night. The total cost was £25. I left the campus at 11:30pm. Can I expense this charge?

        Answer:{{"Classifcation T1" : "Policy Not Violated",
                "Reimbursement Amount" : 25£,
                "Classsifcation T2" : "Fully Reimbursable",
                "Reasons":"The GNEI Expenses Policy states that taxi fares can be claimed for journeys where a member of staff is working in the office very late, specifically after 11pm​​. Therefore, your taxi fare under these circumstances falls within the allowable expenses according to the policy, and total reimbursable amount is £25."}};
        
        <<<<<example three:
        scenario: I booked a taxi from the GNEI campus to my home yesterday night. The total cost was £35. I left the campus at 9:30pm. Can I expense this charge?

        Answer:{{"Classifcation T1" : "Policy Violated",
                "Reimbursement Amount" : 0£,
                "Classsifcation T2" : "Further Clarification Required",
                "Reasons":"Based on the GNEI Expenses Policy, taxi fares can only be claimed for specific reasons, such as if a member of staff is working in the office very late, specifically mentioned as after 11pm. Since you left the campus at 9:30pm, your taxi fare does not meet the criteria set out under the "Taxis" section for allowable taxi expenses.
                Therefore, based on the information provided in the GNEI Expenses Policy, you cannot expense the £35 taxi charge for a trip from the GNEI campus to your home that occurred at 9:30pm."}};
        
        <<<<<example four:
        scenario: I drove to Cambridge for a business trip along with another GNEI colleagues. The total distance covered was 60 miles. I accidentally hit the sidewalk and damaged the side of the car. The total cost of the repairs was £100. How much can I claim in expenses?  

        Answer:{{"Classifcation T1" : "Policy Violated",
                "Reimbursement Amount" : 30£,
                "Classsifcation T2" : "Partially Reimbursable",
                "Reasons":"For your business trip to Cambridge, driving a car for a total distance of 60 miles with another GNEI colleagues, you can claim expenses based on the GNEI Expenses Policy as follows:
                - **Mileage Claim**: According to section **4.32**, mileage for cars is reimbursed at a rate of 50p per mile for the first 11,000 miles in a tax year. Therefore, for 65 miles, the claim would be 60 miles x £0.50/mile = £30.
                - **Damage to the Vehicle**: Section **4.34** specifies that GNEI will not be held liable for damage or repairs to the vehicle used on GNEI’s business. Thus, the cost of repairs due to the accident, totaling £100, cannot be claimed under the policy.
                **Documentation and Approvals Required:**
                1. **Mileage Documentation**: Include details such as the start and end points of the journey, the purpose of the business trip, and the total number of miles claimed.
                2. **Approval for Business Travel**: Ensure that the business trip was pre-approved as per GNEI's travel policy requirements.
                3. **Vehicle Damage**: Since the policy excludes claims for vehicle damage or repairs, there is no requirement for documentation or approval in this context.

                **Total Amount That Can Be Expensed: £30** (for mileage only, as vehicle repair costs are not covered)."}};

        examples End>>>>>

<<<<<Scenario Start:\n\n{question}\n\nScenario End>>>>>

"""

I travelled from London to Amsterdam for a conference on behalf of GNEI for which I the flight tickets cost me £80. Additionally, during this transaction, the bank also charged me £15 for the exchange rate. How much can be reimbursed?
Rate of exchange
If a credit card was used to pay for overseas expenses, the rate charged by the credit card issuer can be claimed. A copy of the statement should be used to evidence the rate used.
For employees GNEIExpenses will use a default rate for translating foreign currency transactions; this can be overridden if different to the rate at the time of expenditure and fluctuations will be allowed within a set tolerance. If the automatic rate is overridden, a copy of the bank statement should be included with the claim to back up the rate used.
Buying foreign currency is not a genuine business expense and cannot be claimed.
--------------------------------------------------------------------------------
Congestion charges
GNEI will not reimburse the co

/home/uceerx0/p3-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
